### importing the libraries

In [1]:

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By

import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select

#installation guide for selenium
#https://medium.com/@shanyitan/how-to-install-selenium-and-run-it-successfully-via-jupyter-lab-c3f50d22a0d4

##### lists for scraper use

In [5]:
hug_by_fac = {'01': ['0255']}

lst_toar = [0,1,2,3,4,5,6,7,8]

#### course number gathering function

In [118]:
#gathering information function
from string import digits
def number_gather(browser):
    lst=[]
    try:
        x=1
        for i in range(0,30):
            try:
                x_pather = '//*[@id="myBody"]/table/tbody/tr/td/table/tbody/tr[5]/td/form/table[1]/tbody/tr[3]/td/a['+str(i)+']/table/tbody/tr/td[1]/table[1]/tbody/tr/td[3]/b'
                main = browser.find_element_by_xpath(x_pather)
                x+=1
                main =''.join(c for c in main if c in digits)
                lst.append(main)
            except:
                continue
    except:
        None
    print(lst)
    return lst

## course numbers scraper
##### creating a dictionary -> maslul:{toar1:{year1:[course1_id,course2_id,course3_id....],year2:[course1_id,course2_id,course3_id....],...},
##### toar2:{year1:[course1_id,course2_id,course3_id....],year2:[course1_id,course2_id,course3_id....],...}}

##### creating the dictionary

In [114]:
maslul_dicto = {}

In [119]:
browser = webdriver.Chrome()
browser.get("https://shnaton.huji.ac.il/")

# lists for checkpoint
hugim_checked = []
faculty_checked = []
maslul_checked = []


for faculty in hug_by_fac.keys():
    faculty_checked.append(faculty)
    try:
        sleep(0.5)
        select = Select(browser.find_element_by_id('faculty'))
        select.select_by_value(str(faculty)) #Pass value as string
        sleep(0.5)

        for hug in hug_by_fac[faculty]: 
            hugim_checked.append(hug)
            sleep(0.5)
            select = Select(browser.find_element_by_id('faculty'))
            select.select_by_value(str(faculty)) #Pass value as string
            sleep(0.5)
            select = Select(browser.find_element_by_id('hugim'))
            select.select_by_value(str(hug))
            sleep(0.5)

            maslul_options = browser.find_elements(By.XPATH,'//*[@name="maslul"]/option')
            #second loop over maslul
            maslul_lst = []

            for mas in maslul_options:
                if mas.get_attribute("value") == "0" or mas.text == "בחרו חוג" or mas.text == "":
                    pass
                else:
                    maslul_lst.append(mas.get_attribute("value")) 
            browser.get("https://shnaton.huji.ac.il/")

            for maslul in maslul_lst:
                try:
                    
                    if maslul is not None:
                        maslul_dicto[maslul]={}
                        maslul_checked.append(maslul)
                        
                    sleep(0.5)
                    select = Select(browser.find_element_by_id('faculty'))
                    select.select_by_value(str(faculty)) #Pass value as string
                    sleep(0.5)

                    select = Select(browser.find_element_by_id('hugim'))
                    select.select_by_value(str(hug))
                    sleep(0.5)

                    select = Select(browser.find_element_by_id('maslul'))
                    select.select_by_value(str(maslul))
                    sleep(0.5)

                    for toar in lst_toar:
                        try:
                            sleep(0.5)
                            maslul_dicto[maslul]={toar:{}}
                            select = Select(browser.find_element_by_id('toar'))
                            select.select_by_value(str(toar))
                            sleep(0.5)
                            for year in range(1,6):
                                select = Select(browser.find_element_by_id('shana'))
                                select.select_by_value(str(year))
                                sleep(0.5)
                                maslul_dicto[maslul][toar][year]:[]
                                print(year)
                                sleep(0.5)

                                browser.find_element_by_css_selector("input[type='radio'][id='full']").click()
                                sleep(0.5)
                                print('pushing')
                                browser.find_element_by_css_selector("input[type='button'][value=' חפש ']").click()
                                
                                #collecting information
                                try:
                                    while True:#information gathering function here
                                        sleep(0.5)
                                        print(number_gather(browser))
                                        for i in number_gather(browser):
                                            print(i)
                                            maslul_dicto[maslul][toar][year].append(i)
                                        browser.find_element_by_xpath("//a[@title='העמוד הבא']").click()
                                except:
                                    browser.get("https://shnaton.huji.ac.il/")

                        except:
                            continue
                except:
                    browser.get("https://shnaton.huji.ac.il/")

    except:
        browser.close()
        print("error in faculty:"+str(faculty)+"hug:"+str(hug)+"maslul:"+str(maslul)+"toar:"+str(toar))
browser.close()

1
pushing
[]
[]
[]
1
pushing
[]
[]
[]


In [120]:
print(maslul_dicto['1500'])

{8: {}}
